In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # ETL Process for Customer 360 Data
# MAGIC This notebook performs an ETL process to create a Customer 360 view by integrating data from various Unity Catalog tables.

# COMMAND ----------

import logging
from pyspark.sql.functions import col, count, max, avg, when, lit, datediff, current_date

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables...")
    policy_df = spark.table("genai_demo.jnj.policy")
    claims_df = spark.table("genai_demo.jnj.claims")
    demographics_df = spark.table("genai_demo.jnj.demographics")
    scores_df = spark.table("genai_demo.jnj.scores")
    aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")

# COMMAND ----------

    # Select relevant fields from demographics data
    logger.info("Selecting relevant fields from demographics data...")
    selected_demographics_df = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", 
        "City", "State", "Postal_Code", "Date_of_Birth", "Gender", 
        "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )

# COMMAND ----------

    # Join demographics with policy data
    logger.info("Joining demographics with policy data...")
    joined_df = selected_demographics_df.join(
        policy_df, selected_demographics_df.Customer_ID == policy_df.customer_id, "inner"
    )

# COMMAND ----------

    # Remove duplicate columns from claims_df before joining
    claims_df = claims_df.drop("Policy_ID")

    # Join the result with claims data
    logger.info("Joining the result with claims data...")
    joined_df = joined_df.join(
        claims_df, joined_df.policy_id == claims_df.Policy_ID, "inner"
    )

# COMMAND ----------

    # Aggregate data
    logger.info("Aggregating data...")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        count("Claim_ID").alias("Total_Claims"),
        count("policy_id").alias("Policy_Count"),
        max("Claim_Date").alias("Recent_Claim_Date"),
        avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Custom calculations
    logger.info("Performing custom calculations...")
    enriched_df = aggregated_df.withColumn(
        "Age", datediff(current_date(), col("Date_of_Birth")) / 365
    ).withColumn(
        "Claim_To_Premium_Ratio", when(col("total_premium_paid") != 0, col("Average_Claim_Amount") / col("total_premium_paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", when(col("Policy_Count") != 0, col("Total_Claims") / col("Policy_Count")).otherwise(0)
    ).withColumn(
        "Retention_Rate", lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", lit("Premium Vehicle Coverage")
    )

# COMMAND ----------

    # Integrate AI/ML insights and customer scores
    logger.info("Integrating AI/ML insights and customer scores...")
    final_df = enriched_df.join(
        aiml_insights_df, "Customer_ID", "inner"
    ).join(
        scores_df, "Customer_ID", "inner"
    )

# COMMAND ----------

    # Write the final DataFrame to a Unity Catalog table
    logger.info("Writing the final DataFrame to a Unity Catalog table...")
    final_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.jnj.customer_360")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process", exc_info=True)
